In [1]:
import win32com.client
import pandas as pd
import os
import time
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch('CpUtil.CpStockCode')
instCpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
instStockChart = win32com.client.Dispatch('CpSysDib.StockChart')


In [4]:
# 크레온 API 연결 확인
# <리턴값 종류 확인 필요>
def check_connection():
    if instCpCybos.IsConnect == 1:
        print('정상적으로 연결')
    else:
        print('연결 실패')
        
check_connection()

연결 실패


In [31]:
# 종목 이름으로 찾기
def search_by_name(stock_name):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(1,i) == stock_name:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_name('삼성전자')

['A005930', '삼성전자']

In [2]:
# 특정 코드의 종목 찾기
def search_by_code(stock_code):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(0,i) == stock_code:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_code('A005930')

['A005930', '삼성전자']

In [6]:
def save_label_stock_info():

    stock_list = os.listdir('../../data_concat/')
    
    for stock in stock_list:

        stock_info = pd.read_csv(f'../../data_concat/{stock}', index_col=0)
        
        concat_list =  []
        concat_list.append('A' + stock.split('_')[-2])

        # for문 1 => 날짜를 반복
        date = stock_info.index.unique()
        stock_info['label'] = 0

        update_stock_info = pd.DataFrame()

        for day in tqdm(date):
            select_day = stock_info.loc[day].copy() # 특정일
            select_day['label'] = 0
            
            # for i in range(len(select_day)):
            for row in range(len(select_day)):
            # for문 2 => 시간을 반복
                pre_price = select_day.iloc[-row-1].values[2] # 선택된 분봉(ROW)의 고가를 가져온다
                # for i in range(len(select_day)-i):
                # print('인덱스 : {0}, 분봉 시간 : {1}'.format(row,select_day.iloc[-row-1].values[0]))
                # print('해당 분봉의 고가 : {0}'.format(pre_price))
                next_price = select_day[-row-1::-1]['고가'].max()
                # print('이후 최대 고가 : {0}'.format(next_price))
                select_day.iloc[-row-1,-1] = next_price
                next_price = 0
            
            update_stock_info = pd.concat([update_stock_info,select_day])
        for code in concat_list:
            update_stock_info.to_csv('../../data_label/label_{0}_{1}.csv'.format(
                search_by_code(code)[0][1:],search_by_code(code)[1]
                ), encoding='utf-8-sig')

In [7]:
save_label_stock_info()

100%|██████████| 429/429 [00:23<00:00, 18.52it/s]


In [7]:
# 종목명 전체 리스트 가져오기
# <선물이랑 ETF 쪽은 구분 필요>
def get_stock_list():
    count = instCpStockCode.GetCount()
    stock_list = []
    
    for i in range(count):
        stock_list.append(instCpStockCode.GetData(1,i))
    
    return stock_list

# get_stock_list()

In [8]:
stock_list = []
stock_list = stock_list + list(instCpCodeMgr.GetStockListByMarket(2))
stock_list_real = []
p = re.compile('^A')

for stock in stock_list:
        if p.match(str(stock)) != None:
                stock_list_real.append(stock)

NameError: name 're' is not defined

In [9]:
# print(instCpCodeMgr.GetStockListByMarket(1))

In [9]:
# 전체 종목 가져오기
codelist = instCpCodeMgr.GetStockListByMarket(1)
kospi = {}
for code in codelist:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name
    
# kospi

In [11]:
# 
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    # print(i, code, secondCode, name)

In [15]:
code_name_list = [] 
code_list = []
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    code_list.append(code)
    code_name_list.append(name)
# print(code_list)
# print(code_name_list)


In [ ]:

clist = []
for code in code_list:
    clist.append(search_by_code(code))

In [16]:
clist

[['A000020', '동화약품'],
 ['A000040', 'KR모터스'],
 ['A000050', '경방'],
 ['A000060', '메리츠화재'],
 ['A000070', '삼양홀딩스'],
 ['A000075', '삼양홀딩스우'],
 ['A000080', '하이트진로'],
 ['A000087', '하이트진로2우B'],
 ['A000100', '유한양행'],
 ['A000105', '유한양행우'],
 ['A000120', 'CJ대한통운'],
 ['A000140', '하이트진로홀딩스'],
 ['A000145', '하이트진로홀딩스우'],
 ['A000150', '두산'],
 ['A000155', '두산우'],
 ['A000157', '두산2우B'],
 ['A000180', '성창기업지주'],
 ['A000210', 'DL'],
 ['A000215', 'DL우'],
 ['A000220', '유유제약'],
 ['A000225', '유유제약1우'],
 ['A000227', '유유제약2우B'],
 ['A000230', '일동홀딩스'],
 ['A000240', '한국앤컴퍼니'],
 ['A000270', '기아'],
 ['A000300', '대유플러스'],
 ['A000320', '노루홀딩스'],
 ['A000325', '노루홀딩스우'],
 ['A000370', '한화손해보험'],
 ['A000390', '삼화페인트'],
 ['A000400', '롯데손해보험'],
 ['A000430', '대원강업'],
 ['A000480', '조선내화'],
 ['A000490', '대동'],
 ['A000500', '가온전선'],
 ['A000520', '삼일제약'],
 ['A000540', '흥국화재'],
 ['A000545', '흥국화재우'],
 ['A000547', '흥국화재2우B'],
 ['A000590', 'CS홀딩스'],
 ['A000640', '동아쏘시오홀딩스'],
 ['A000650', '천일고속'],
 ['A000660', 'SK하이닉스'],
 ['A000670', 

In [10]:
import re
 
def clean_text(inputString):
  text_rmv = re.sub('Q.+', '', inputString)
  return text_rmv

def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()
result = listToString(code_list)

str =clean_text(result)
# print(str)
my_list = str.split(' ')
# print(my_list)

In [20]:
files_m = os.listdir('../data')
files_d = os.listdir('../data_day')
files_m

['000660_SK하이닉스.csv']

In [21]:
files_d

['day_000660_SK하이닉스.csv']

In [22]:
len(files_m)

1

In [24]:
globals()['data_m0'].index

Int64Index([20221101, 20221101, 20221101, 20221101, 20221101, 20221101,
            20221101, 20221101, 20221101, 20221101,
            ...
            20201023, 20201023, 20201023, 20201023, 20201023, 20201023,
            20201023, 20201023, 20201023, 20201023],
           dtype='int64', length=117626)

In [25]:
data_d0.head(1)

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221101,0,82700,83900,81400,83900,1200,3820185,316974000000,1613467,728002000,...,50.75,20221101,100.0,56046,-30105835,0,0.0,0,0.0,0.0


In [48]:
# 분봉 일봉 합치기
def code_concat():
    files_m = os.listdir('../data')
    files_d = os.listdir('../data_day') 
    
    for j, (min_df, day_df) in enumerate(zip(files_m, files_d)):
      globals()[f'data_m{j}'] = pd.read_csv(f'../data/{min_df}', index_col='Unnamed: 0')
      globals()[f'data_d{j}'] = pd.read_csv(f'../data_day/{day_df}', index_col='Unnamed: 0')
      
      for v in tqdm(globals()[f'data_m{j}'].index):
            input_list = ['전일대비','상장주식수','시가총액','외국인주문한도수량'
                    ,'외국인주문가능수량','외국인현보유수량','외국인현보유비율'
                    ,'수정주가일자','수정주가비율','기관순매수량','기관누적순매수량']
            
            globals()[f'data_m{j}'].loc[v, input_list] = globals()[f'data_d{j}'].loc[v, input_list].values
           

In [ ]:
def code_concat_save(concat_list):
        for c, code in tqdm(enumerate(concat_list)):
                globals()[f'data_m{c}'].to_csv('../data_concat/concat_{0}_{1}.csv'.format(
                                search_by_code(code)[0],search_by_code(code)[1]
                        ), encoding='utf-8-sig')

In [46]:
files_m = os.listdir('../data')
files_d = os.listdir('../data_day') 
    
for j, (min_df, day_df) in enumerate(zip(files_m, files_d)):
    globals()[f'data_m{j}'] = pd.read_csv(f'../data/{min_df}', index_col='Unnamed: 0')
    globals()[f'data_d{j}'] = pd.read_csv(f'../data_day/{day_df}', index_col='Unnamed: 0')


In [12]:
# 분봉 일봉 합친 것 저장
def code_concat_save(concat_list):
        for c, code in tqdm(enumerate(concat_list)):
                globals()[f'data_m{c}'].to_csv('../data_concat/concat_{0}_{1}.csv'.format(
                                search_by_code(code)[0],search_by_code(code)[1]
                        ), encoding='utf-8-sig')
                

In [29]:
test_list =['A000660']
# code_concat(test_list)

In [34]:
code_concat_save(test_list)

1it [00:00,  1.20it/s]


In [13]:
pd.set_option('display.max_columns', None)

In [16]:
import time
# 특정 범위 일자 종목 데이터 가져오기
def get_stock_info(stock_code, start_day, end_day, type):
    
    instStockChart.SetInputValue(0, stock_code) # 종목명
    instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, start_day) # 요청 시작일
    instStockChart.SetInputValue(2, end_day) # 요청 종료일
    # instStockChart.SetInputValue(4, request_num) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 8 : 거래량, 9 : 거래대금, 13 : 시가총액
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
    instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
    
    return stock_info

donghwa = get_stock_info('A005930', '20221029', '20221031', 'm')

com_error: (-2147352567, '예외가 발생했습니다.', (0, None, 'U-CYBOS가 서버에 접속되어 있지 않습니다.', None, 0, -2147483638), None)

In [15]:
import time
# 전체 일자 분봉 종목 데이터 가져오기
def get_stock_info_auto(stock_code, end_day, type):
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=2)
    minus2_day = timedelta(days=1)
    
    transfer_end_day = datetime.strptime(end_day, day_format) # end_day 변환
    pre_day = datetime.strftime(transfer_end_day - minus_day, day_format) # pre_day 계산 => '20221028'

    stock_df = get_stock_info(stock_code, pre_day, end_day, type) # 첫번째 DF 생성 => pre_day '20221028' end_day '20221103'
    
    stock_name = search_by_code(stock_code)

    for day in range(370): # range 범위 수정하지 말 것
        transfer_pre_day = datetime.strptime(pre_day, day_format) # '20221028' '20221023'
        next_pre_day = datetime.strftime(transfer_pre_day - minus_day, day_format) # '20221023' '20221018'
        pre_day = datetime.strftime(transfer_pre_day - minus2_day, day_format) # '20221027' '20221022'

        info_second = get_stock_info(stock_code, next_pre_day, pre_day, type) # '20221023' '20221027'

        time.sleep(1)
        print(next_pre_day, pre_day)

        pre_day = next_pre_day # '20221027' =  '20221023'
        
        stock_df = pd.concat([stock_df, info_second], axis=0)
        info_second = pd.DataFrame()
        print(stock_df.shape)
        print(stock_name, 'Count : {0}, 남은 요청 횟수 :{1}'.format(day, instCpCybos.GetLimitRemainCount(1))) 
    
    stock_df.to_csv('../data/{0}_{1}.csv'.format(stock_name[0][1:],stock_name[1]), encoding='utf-8-sig')

    return stock_df

# samsung = get_stock_info_auto('A005930', '20221101', 'm')
# samsung

In [ ]:
import time
for code in code_list[:]:
    get_stock_info_auto(code, '20221103', 'm')


In [17]:
test = pd.DataFrame()
test.shape[]

0

In [95]:
get_stock_info_auto('A000660', '20221103', 'd')

20221030 20221031
(1180, 24)
['A005930', '삼성전자'] Count : 0, 남은 요청 횟수 :58
20221028 20221029
(1561, 24)
['A005930', '삼성전자'] Count : 1, 남은 요청 횟수 :57
20221026 20221027
(2323, 24)
['A005930', '삼성전자'] Count : 2, 남은 요청 횟수 :56
20221024 20221025
(3085, 24)
['A005930', '삼성전자'] Count : 3, 남은 요청 횟수 :55
20221022 20221023
(3085, 24)
['A005930', '삼성전자'] Count : 4, 남은 요청 횟수 :54
20221020 20221021
(3847, 24)
['A005930', '삼성전자'] Count : 5, 남은 요청 횟수 :53
20221018 20221019
(4609, 24)
['A005930', '삼성전자'] Count : 6, 남은 요청 횟수 :52
20221016 20221017
(4990, 24)
['A005930', '삼성전자'] Count : 7, 남은 요청 횟수 :51
20221014 20221015
(5371, 24)
['A005930', '삼성전자'] Count : 8, 남은 요청 횟수 :60
20221012 20221013
(6133, 24)
['A005930', '삼성전자'] Count : 9, 남은 요청 횟수 :59
20221010 20221011
(6514, 24)
['A005930', '삼성전자'] Count : 10, 남은 요청 횟수 :58
20221008 20221009
(6514, 24)
['A005930', '삼성전자'] Count : 11, 남은 요청 횟수 :57
20221006 20221007
(7276, 24)
['A005930', '삼성전자'] Count : 12, 남은 요청 횟수 :56
20221004 20221005
(8038, 24)
['A005930', '삼성전자']

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221103,1530,59200,59200,59200,59200,0,1859760,110097790000,5944330,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1520,59500,59700,59500,59700,0,193177,11510350000,5944330,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1519,59500,59600,59500,59600,0,49046,2921250000,5893859,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1518,59600,59600,59500,59600,0,20663,1230500000,5874904,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221103,1517,59500,59600,59500,59600,0,20021,1192690000,5864749,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,59900,60000,59800,59800,0,450531,26982330000,730219,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,904,59900,60000,59900,59900,0,159408,9552240000,494948,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,903,60000,60000,59900,59900,0,41872,2509660000,372455,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,902,59900,60100,59900,60000,0,172825,10367550000,345917,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0


In [36]:
test.index[-1]

'20221102'

In [96]:
test = pd.read_csv('../data/005930_삼성전자.csv', encoding='utf-8-sig')
test['Unnamed: 0'].value_counts()

20221103    381
20210705    381
20210616    381
20210617    381
20210618    381
           ... 
20220314    381
20201027    381
20210104    321
20220103    321
20221101     37
Name: Unnamed: 0, Length: 500, dtype: int64